## 0. Import Dependencies

In [ ]:
!pip install tweepy
!pip install ibm_watson

## 1. Get Data from Twitter

In [ ]:
# Import os to work with the operating system
import os 

# Import tweepy to work with the twitter API
import tweepy as tw

# Import pandas to work with dataframes
import pandas as pd

In [ ]:
consumer_key = 'YOUR TWITTER CONSUMER KEY'
consumer_secret = 'YOUR TWITTER CONSUMER SECRET'
access_token = 'YOUR TWITTER ACCESS TOKEN'
access_token_secret = 'YOUR TWITTER ACCESS TOKEN SECRET'

In [ ]:
# Authenticate
auth = tw.OAuthHandler(consumer_key, consumer_secret)
# Set Tokens
auth.set_access_token(access_token, access_token_secret)
# Instantiate API
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
handle = 'TiffanyHaddish'

In [ ]:
res = api.user_timeline(screen_name=handle, count=100, include_rts=True)

In [ ]:
tweets = [tweet.text for tweet in res]

In [ ]:
tweets

In [ ]:
text = ''.join(str(tweet) for tweet in tweets)

In [ ]:
text

## 2. Setup Personality Insights

In [ ]:
# Import Watson
from ibm_watson import PersonalityInsightsV3

# Import authenticator
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
# Creds for Personality Insights
apikey = 'YOUR PERSONALITY INSIGHTS API KEY'
url = 'YOUR PERSONALITY INSIGHTS URL'

In [ ]:
# Authenticate to PI service
authenticator = IAMAuthenticator(apikey)
personality_insights = PersonalityInsightsV3(
        version='2017-10-13', 
        authenticator=authenticator
)
personality_insights.set_service_url(url)

In [ ]:
profile = personality_insights.profile(text, accept='application/json').get_result()
profile

In [ ]:
for personality in profile['personality']:
    print(personality['name'], personality['percentile'])

In [ ]:
for personality in profile['values']:
    print(personality['name'], personality['percentile'])

In [ ]:
for personality in profile['needs']:
    print(personality['name'], personality['percentile'])

## 3. Visualise the Results

In [ ]:
# Import matplotlib 
from matplotlib import pyplot as plt
# Import seaborn
import seaborn as sns

In [ ]:
# Visualise profiles 
needs = profile['needs']
result = {need['name']:need['percentile'] for need in needs}
df = pd.DataFrame.from_dict(result, orient='index')
df.reset_index(inplace=True)
df.columns = ['need', 'percentile']

In [ ]:
df.head()

In [ ]:
# Create Plot
plt.figure(figsize=(15,5))
sns.barplot(y='percentile', x='need', data=df).set_title('Needs')
plt.show()

In [ ]:
# Create plotting Function
def create_trait_plot(name, traits):
    result = {trait['name']:trait['percentile'] for trait in traits}
    df=pd.DataFrame.from_dict(result, orient='index')
    df.reset_index(inplace=True)
    df.columns = ['need', 'percentile']
    plt.figure(figsize=(15,5))
    sns.barplot(y='percentile', x='need', data=df).set_title(name)
    plt.show()

In [ ]:
[create_trait_plot(personality_trait['name'], personality_trait['children']) for personality_trait in profile['personality']]